In [ ]:
%%capture output
# !pip install requests selenium anytree
# !apt install chromium-chromedriver
from importlib import import_module
import os
os.system('apt install chromium-chromedriver')
for module in ['selenium', 'bs4', 'anytree', 'requests', 'webdriver_manager', 'pymysql', 'pydrive']:
    try:
        import_module(module)
    except ImportError:
        os.system('pip install {}'.format(module))

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.request import urlopen
from anytree import Node, RenderTree
from googleapiclient.discovery import build
import pymysql.cursors 
import six
import traceback
import requests
import re
import time
import os
import shutil
import cv2
import youtube_dl
import pandas as pd
from random import randint
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [ ]:
url = # put your FIRST_VIDEO_URL
path = # path to directory where you will store screenshots from video

#------------------- YT API configuration ----------------#
DEVELOPER_KEY = # YouTube API developer key
youtube = build('youtube', 'v3', developerKey=DEVELOPER_KEY)
#---------------------------------------------------------#

#--------------------- DB connection ---------------------#
conn = pymysql.connect(host= #host_number,
							 user= #use_name,
							 port= #port_number,
							 password= #password,
							 db= #database_name,
							 unix_socket='/tmp/mysql.sock',
							 autocommit=True)
#---------------------------------------------------------#


#------------------- GD API configuration ----------------#
gauth = GoogleAuth() 										
drive = GoogleDrive(gauth) 									
#---------------------------------------------------------#

def capture_frame(url, video_id, path):

    if os.path.isdir(path): shutil.rmtree(path)
    os.mkdir(path)

    shared_folder_id = # GOOGLE_DRIVE_ID

    ydl_opts={}
    ydl=youtube_dl.YoutubeDL(ydl_opts)
    info_dict=ydl.extract_info(url, download=False)
    count = 0
    formats = info_dict.get('formats',None)
    for f in formats:
        if f.get('format_note', None) == '144p' and count < 1:
            url_new = f.get('url',None)
            cap = cv2.VideoCapture(url_new)
            total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
            nth_frames = sorted([randint(0,total_frames) for p in range(10)])
            folder = drive.CreateFile({"title": url.split('watch?v=')[1], 
                                       "parents": [{'id':shared_folder_id}], 
                                       "mimeType":"application/vnd.google-apps.folder"})
            file_list = drive.ListFile({'q': "'YOUR_DRIVE_ID' in parents and trashed=false"}).GetList()
            file_list = set([file['title'] for file in file_list])
           
            if video_id in file_list: continue
                
            folder.Upload()
            x=0
            while x<10:
                ret, frame = cap.read()
                if not ret:
                    print("NOO")
                    break
                filename = path + str(x) + ".png"
                cv2.imwrite(filename, frame)
                d = drive.CreateFile({"parents": [{"kind": "drive#fileLink","id": folder['id']}]})
                d.SetContentFile(filename)
                d.Upload()
                cap.set(1,nth_frames[x])
                x+=1
                if cv2.waitKey(30)&0xFF == ord('q'):
                    break
            count += 1
            cap.release()
    shutil.rmtree(path)

    
def scroll_down(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
        # Wait to load page
        time.sleep(1)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


def get_recommended_vids(url, dictionary):
    yt = 'https://www.youtube.com'
    options = Options()
    options.headless = True
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("window-size=1920,1080")
    #driver = webdriver.Chrome('chromedriver', options=options)
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

    driver.get(url)
    '''
    scroll_down(driver) ######## UNCOMMENT THIS
    '''
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    res = soup.select('a.yt-simple-endpoint.style-scope.ytd-compact-video-renderer')
    dictionary['rec_vids'] = [yt + i.get('href') for i in res] 
    print('URL: ', url)
    #dictionary['rec_vids'] = dictionary['rec_vids'][0:3] ### DELETE THIS

    driver.close()  
    return dictionary


def getID(url):
    # if shortlink
    if "youtu.be" in url: 
        vid_id = url.split("/")[-1]
    # standard URL
    else: 
        vid_id = url.split("v=")[1]
    return vid_id


def get_vid_details(youtube, **kwargs):
    return youtube.videos().list(part="snippet, contentDetails, statistics, status", **kwargs).execute()


def store_video_infos(video_response, dictionary, video_id):
    try: items = video_response.get("items")[0] 
    except Exception: items = None
    # get the snippet, statistics & content details from the video response
    try: snippet         = items["snippet"] 
    except Exception: snippet = None
    try: statistics      = items["statistics"] 
    except Exception: statistics = None
    try: content_details = items["contentDetails"] 
    except Exception: content_details = None
    try: status          = items["status"] 
    except Exception: status = None
    # get infos from the snippet
    try: channel_title = snippet["channelTitle"] 
    except Exception: channel_title = None
    try: title         = snippet["title"] 
    except Exception: title = None
    try: description   = snippet["description"] 
    except Exception: description = None
    try: publish_time  = snippet["publishedAt"] 
    except Exception: publish_time = None
    try: keywords      = snippet["tags"] 
    except Exception: keywords = None
    # get status
    try: forKids       = status['madeForKids'] 
    except Exception: forKids = None
    # get stats infos 
    try: comment_count = statistics["commentCount"] 
    except Exception: comment_count = None
    try: like_count    = statistics["likeCount"] 
    except Exception: like_count = None
    try: dislike_count = statistics["dislikeCount"] 
    except Exception: dislike_count = None
    try: view_count    = statistics["viewCount"] 
    except Exception: view_count = None
    # get duration from content details
    try: duration = content_details["duration"] 
    except Exception: duration = None
    # duration in the form of something like 'PT5H50M15S'
    # parsing it to be something like '5:50:15'
    # print('duration: ', duration)
    try: parsed_duration = re.search(f"PT(\d+H)?(\d+M)?(\d+S)", duration).groups() 
    except Exception: parsed_duration = None
    if parsed_duration != None:
        duration_str = ""
        for d in parsed_duration:
            if d: duration_str += f"{d[:-1]}:"
        duration_str = duration_str.strip(":")
    else:
        duration_str = parsed_duration
    
    publish_time = publish_time.replace('T',' ')
    publish_time = publish_time.replace('Z',' ')
    # print(f"""\
    # Title: {title}
    # Description: {description}
    # Channel Title: {channel_title}
    # Publish time: {publish_time}
    # Duration: {duration_str}
    # Number of comments: {comment_count}
    # Number of likes: {like_count}
    # Number of dislikes: {dislike_count}
    # Number of views: {view_count}
    # Keywords: {keywords}
    # forKids?: {forKids}
    # """)

    dictionary['id'] = video_id
    dictionary['title'] = title
    dictionary['description'] = description
    dictionary['channel_title'] = channel_title
    dictionary['publish_time'] = publish_time
    dictionary['duration'] = duration_str
    dictionary['comment_count'] = int(comment_count) if comment_count != None else None
    dictionary['like_count'] = int(like_count) if like_count != None else None
    dictionary['dislike_count'] = int(dislike_count) if dislike_count != None else None
    dictionary['view_count'] = int(view_count)
    dictionary['tags'] = None if keywords == None else ', '.join(keywords)
    dictionary['forKids'] = 1 if forKids == True else 0

    return dictionary


def limitVisit(df, video_id):
    cur = conn.cursor()
    if cur.execute("SELECT * FROM yt_video WHERE visits = 10 AND id = %s;", video_id):
        print("\n* Limiting visit to this video *\n")
        return 1


def db_store(dictionary):
    with conn.cursor() as cursor:
        try:
            cursor.execute("SET FOREIGN_KEY_CHECKS=0")
            sql = "INSERT INTO yt_video (id, title, description, channel_title, publish_time, duration, comment_count, like_count, dislike_count, view_count, tags, forKids, rec_vids, parent) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE visits = IF(visits < 10, visits + 1, visits);"
            # print('\n\n%s\n, %s\n, %s\n, %s\n, %s\n, %s\n, %d\n, %d\n, %d\n, %d\n, %s\n, %d\n, %s\n, %s\n\n'%(dictionary['id'], dictionary['title'], dictionary['description'], dictionary['channel_title'], dictionary['publish_time'], dictionary['duration'], dictionary['comment_count'], dictionary['like_count'], dictionary['dislike_count'], dictionary['view_count'], dictionary['tags'], dictionary['forKids'], ', '.join(dictionary['rec_vids']), dictionary['parent']))
            # print(type(dictionary['id']), type(dictionary['title']), type(dictionary['description']), type(dictionary['channel_title']), type(dictionary['publish_time']), type(dictionary['duration']), type(dictionary['comment_count']), type(dictionary['like_count']), type(dictionary['dislike_count']), type(dictionary['view_count']), type(dictionary['tags']), type(dictionary['forKids']), type(', '.join(dictionary['rec_vids'])), type(dictionary['parent']))
            arg = (dictionary['id'], dictionary['title'], dictionary['description'], dictionary['channel_title'], dictionary['publish_time'], dictionary['duration'], dictionary['comment_count'], dictionary['like_count'], dictionary['dislike_count'], dictionary['view_count'], dictionary['tags'], dictionary['forKids'], ', '.join(dictionary['rec_vids']), dictionary['parent']) 
            print('###################')
            print('PARENT: ', dictionary['id'])
            for vid_id in dictionary['rec_vids']:
                vid_id = vid_id.split("/watch?v=")[1]                
                print('---> CHILD: ', vid_id)
                sql2 = "INSERT INTO recommended (parent, rec_vid) VALUES (%s, %s);"
                arg2 = (dictionary['id'], vid_id)
                cursor.execute(sql2, arg2)
            print('###################\n')
            cursor.execute(sql, arg)
            conn.commit()
            cursor.execute("SET FOREIGN_KEY_CHECKS=1")
            print("CURSOR EXECUTED!!\n")
            conn.commit()
        except Exception as err:
            print("Something went wrong: {}".format(err))
            traceback.print_exc()
        
'''
sql2 = "INSERT INTO recommended (parent, rec_vid) VALUES (%s, %s);"
arg2 = (dictionary['parent'], )
'''

vids_info_list = []
first_vid_id = getID(url)
column_names = ['id', 'title', 'description', 'channel_title', 'publish_time', 
                'duration', 'comment_count', 'like_count', 'dislike_count', 
                'view_count', 'tags', 'forKids', 'rec_vids', 'parent']
df = pd.DataFrame(columns = column_names)


# Return recommended videos + append video info to the list
def gather_vids(url, youtube, df, parent = None, path=path):
    info = {}
    video_id = getID(url)
    if limitVisit(df, video_id) == 1:
        pass
    else:
        capture_frame(url, video_id, path)
        results = youtube.videos().list(id=video_id, part='snippet, contentDetails, statistics, status').execute()
        info = store_video_infos(results, info, video_id)
        info = get_recommended_vids(url, info)
        info['parent'] = None if parent==None else str(parent)
        print('VIDEO REC: ', info['rec_vids'])
        df = df.append(info, ignore_index=True)
        db_store(info)
    return df



idx = 0
df = gather_vids(url, youtube, df)
while True:
    print('idx: ', idx)
    try:
        next_rec_vid = df.iloc[0]['rec_vids'][idx]
    except:
        df.iloc[0]['rec_vids'] = get_recommended_vids("https://www.youtube.com/watch?v="+df.iloc[0]['id'], {'rec_vids': []})['rec_vids']
        next_rec_vid = df.iloc[0]['rec_vids'][idx]
    parent_id = df.iloc[0]['id']
    df = gather_vids(next_rec_vid, youtube, df, parent_id)
    print('check df here: \n', df)
    idx += 1
    if idx == len(df.iloc[0]['rec_vids']):
        idx = 0
        df = df.iloc[1: , :]



'''
Chrome driver runs in option headless
Seems like the recommended videos are higly repetitive within the network 
when run this way.

Seems to give a more diverse list of suggestions 
when links are manually open in incognito mode
'''
